# <center> **Transformer - Attention Is All You Need !!!**  </center>

<div class="row">
  <div class="column" style = " float: center; width: 100%; padding: 5px">
    <img src="https://www.analyticsinsight.net/wp-content/uploads/2022/05/Know-About-Transformer-Machine-learning-Model-at-a-Glance-1440x564_c.jpg" alt="Positional Embedding in Encoder" style="height:100%">
  </div>
</div>

### Download Necessary Packages

In [50]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam

import nltk
from collections import Counter
import contractions
import re
from nltk import sent_tokenize
from nltk.corpus import stopwords
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.preprocessing.sequence import pad_sequences


### Connect Google Colab TPU

In [5]:
os.environ['TPU_NAME']

'grpc://10.50.101.114:8470'

In [6]:
tpu_address = os.environ['TPU_NAME']
tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_address)
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

strategy = tf.distribute.experimental.TPUStrategy(tpu)

print(f"TPU with address-  {tpu.cluster_spec().as_dict()['worker']} successfully connected")
print(f"Number Of Accelerators- {strategy.num_replicas_in_sync}")

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.50.101.114:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.50.101.114:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


TPU with address-  ['10.50.101.114:8470'] successfully connected
Number Of Accelerators- 8


### Dataset

In [60]:
EMBEDDING_MATRIX = np.load('./Embedding_Matrix.npy')
X_train = np.load('./X_train.npy')
Y_train = np.load('./Y_train.npy')
X_val = np.load('./X_val.npy')
Y_val = np.load('./Y_val.npy')

In [61]:
Y_train = Y_train.reshape((-1, 1))
Y_val = Y_val.reshape((-1, 1))

In [62]:
print(type(X_train))
print(X_train.shape, Y_train.shape)
print(X_val.shape, Y_val.shape)

<class 'numpy.ndarray'>
(30000, 1024) (30000, 1)
(10000, 1024) (10000, 1)


In [63]:
BATCH_SIZE = 128
MAX_SEQ_LEN = 1024
EMB_DIMS = 300
VOCAB_SIZE = EMBEDDING_MATRIX.shape[0]

In [64]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(30000).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, Y_val)).shuffle(10000).take(5000)

## <center style = 'font-family:"courier"'> **Transformer Encoder** </center>
<center>
    <img src = "https://www.researchgate.net/publication/334288604/figure/fig1/AS:778232232148992@1562556431066/The-Transformer-encoder-structure.ppm"> </img>
    </cener>

<p align = "justify" style = 'font-family = "courier"'>Simple RNN Networks like GRUs or LSTMs do not work very well for long sentences. Simply put forward, it is difficult to keep all the information learnt in a single vector (the activation vector, **a** of the model for long sentences. After certain timesteps, the previous information tends to be lost, while current ones being captured. To address this issue, **Attention Mechanism** was introduced which took the NLP research with a storm. Everything was about **Attention** then.
In 2017 a research paper was published, namely **Attention Is All You Need**. This was when Transformers came into existance. Transformers tried to combine the characteristics of **Convolutional Neural Networks and Attention Mechanism**. 
One main difference  between other architectures and transformer is that the **input sequence can be passed parallelly** so that GPU can be used effectively and the speed of training can also be increased. It is also based on the multi-headed attention layer, so it easily overcomes the vanishing gradient issue. Transformers made **Vectorization** possible for working on Text Data and thus increasing the speed of of training.</p>

### Positional Embeddings
<p align = "justify">In the earlier approach, for sequential models, inputs were fed in sequence, so the effect of positions were automatically captured by the learning algorithm. But, in **Transformer Architecture**, inputs were all fed at once, so making it difficult for the architecture to guess the impact of position of a word in the sentence/paragraph. Thus, **Positional Encodings** were brought in.
The Inputs after being converted into Embedded Matrices are added with the positional embedding matrix for the architecture to have knowledge about the position of the words.

    
<div class="row">
  <div class="column" style = " float: left; width: 30%; padding: 5px">
    <img src="https://vaclavkosar.com/images/transformer-positional-embeddings.png" alt="Positional Embedding in Encoder">
  </div>
  <div class="column" style = "float: left; width: 50%; padding: 5px">
    <img src="https://jinglescode.github.io/assets/img/posts/illustrated-guide-transformer-10.jpg" alt="Embeddings working">
  </div>
</div>

In [65]:
def get_positional_embedding(num_positions:int, dimensions:int):
    """
    num_positions: Length Of Sequences in the dataset after padding
    dimensions: Number of dimensions used to represent each word in embedding matrix
    """
    # Create a column vector for positions
    pos_vec = np.arange(num_positions)[:, np.newaxis]
    
    # Create a row vector for dimensions
    dims_vec = np.arange(dimensions)[np.newaxis, :]
    
    i = pos_vec // 2
    angles = pos_vec * 1.0 / (pow(10_000, 2 * i / dimensions))
    angles[:, 0::2] = np.sin(angles[:, 0::2])
    angles[:, 0::1] = np.cos(angles[:, 0::1])
    pos_encoding = angles[np.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

In [17]:
get_positional_embedding(10, 4)

<tf.Tensor: shape=(1, 10, 1), dtype=float32, numpy=
array([[[1.        ],
        [0.66636676],
        [0.9998    ],
        [0.99955016],
        [0.99999994],
        [0.9999999 ],
        [1.        ],
        [1.        ],
        [1.        ],
        [1.        ]]], dtype=float32)>

### Encoder Layer

In [66]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, embedding_dims, num_heads,
                  fully_connected_dim, dropout_rate = 0.1, 
                  layernorm_eps = 1e-6):
        super(EncoderLayer, self).__init__()
        
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads,
                                                      key_dim = embedding_dims,
                                                      dropout = dropout_rate)
        
        self.ffn = tf.keras.Sequential([
        tf.keras.layers.Dense(fully_connected_dim, activation='relu'),
        tf.keras.layers.Dense(embedding_dims)])
        
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon = layernorm_eps)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon = layernorm_eps)
        self.dropout_ffn = tf.keras.layers.Dropout(dropout_rate)
        
    def call(self, x, training = False):
        attn_out = self.mha(x, x, x, training = training)
        attn_out = self.layernorm1(x + attn_out)
        ffn_out = self.ffn(attn_out)
        ffn_out = self.layernorm2(attn_out + ffn_out)
        return ffn_out

### Encoder

In [67]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, input_vocab_size, embedding_dims, num_heads,
                 fully_connected_dim, max_position_encoding, pretrained_embedding_matrix = None,
                 dropout_rate = 0.1, layernorm_eps = 1e-6):
        super(Encoder, self).__init__()
        
        self.embedding_dims = embedding_dims
        
        
        self.embeddings = tf.keras.layers.Embedding(input_vocab_size, self.embedding_dims)
        if pretrained_embedding_matrix is not None:
            self.embeddings.embeddings_initializer = tf.keras.initializers.constant(pretrained_embedding_matrix)
            self.embeddings.trainable = False
        
        self.positional_encoding = get_positional_embedding(max_position_encoding, self.embedding_dims)
        self.encoding_layer = EncoderLayer(embedding_dims, num_heads,
                  fully_connected_dim, dropout_rate = dropout_rate, 
                  layernorm_eps = layernorm_eps)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
    
    def call(self, x, training = False):
        seq_len = tf.shape(x)[1]
        
        x = self.embeddings(x)
        
        ## Scaling the embeddings
        x *= tf.math.sqrt(tf.cast(self.embedding_dims, tf.float32))
        
        ## Adding the Positional Encoding to embeddings
        x += self.positional_encoding[:, :seq_len, :]
        x = self.dropout(x, training = training)
        
        x = self.encoding_layer(x, training = training)
        
        return x

In [68]:
class FullyConnectedLayer(tf.keras.layers.Layer):

    def __init__(self):
        super(FullyConnectedLayer, self).__init__()

        self.conv1 = tf.keras.layers.Conv1D(64, kernel_size = 8, strides = 4)
        self.conv2 = tf.keras.layers.Conv1D(128, kernel_size = 8, strides = 4)
        self.conv3 = tf.keras.layers.Conv1D(256, kernel_size = 8, strides = 4)

        self.maxpool1 = tf.keras.layers.MaxPooling1D()
        self.maxpool2 = tf.keras.layers.MaxPooling1D()
        self.maxpool3 = tf.keras.layers.MaxPooling1D()

        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(32, activation = 'relu')
        self.output_layer = tf.keras.layers.Dense(1, activation = 'sigmoid')
        
    def call(self, encoder_out):
        x = self.conv1(encoder_out)
        x = self.maxpool1(x)

        x = self.conv2(x)
        x = self.maxpool2(x)

        x = self.conv3(x)
        x = self.maxpool3(x)

        x = self.flatten(x)
        x = self.dense1(x)
        x = self.output_layer(x)
        return x

### Machine Learning Model

In [69]:
def Mymodel(input_shape, input_vocab_size, embedding_dims, embedding_matrix = None):
    input_x = tf.keras.layers.Input(shape = input_shape)
    encoder_out = Encoder(input_vocab_size = input_vocab_size,
                      embedding_dims = embedding_dims,
                      num_heads = 5,
                     fully_connected_dim = 128,
                      max_position_encoding = input_shape, pretrained_embedding_matrix = embedding_matrix,
                     dropout_rate = 0.1, layernorm_eps = 1e-6
        )(input_x)

    outputs = FullyConnectedLayer()(encoder_out)
    model = tf.keras.Model(inputs = input_x, outputs = outputs)
    return model

### Create Training Loop

In [70]:
with strategy.scope():
    model = Mymodel(MAX_SEQ_LEN, VOCAB_SIZE, EMB_DIMS, EMBEDDING_MATRIX)

    loss_object = tf.keras.losses.BinaryCrossentropy(reduction = tf.keras.losses.Reduction.NONE)

    def compute_loss(labels, predictions):
        per_example_loss = loss_object(labels, predictions)
        return tf.nn.compute_average_loss(per_example_loss, global_batch_size = BATCH_SIZE * strategy.num_replicas_in_sync)

    test_loss = tf.keras.metrics.Mean(name = "test_loss")
    train_accuracy = tf.keras.metrics.BinaryAccuracy(name = "train_accuracy")
    test_accuracy = tf.keras.metrics.BinaryAccuracy(name = "test_accuracy")

    lr_scheduler = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate = 0.001,
        decay_steps =500,
        decay_rate = 0.9
    )

    optimizer = tf.keras.optimizers.Adam(learning_rate = lr_scheduler)

    @tf.function
    def distributed_training_step(datasets_inputs):
        per_replica_losses = strategy.run(train_steps, args = (datasets_inputs, ))
        print(per_replica_losses)
        return strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses, axis = None)
    
    @tf.function
    def distributed_test_step(datasets_inputs):
        strategy.run(test_steps, args = (datasets_inputs, ))
    
    
    def train_steps(inputs):
        text_seq, labels = inputs

        with tf.GradientTape() as tape:
            predictions = model(text_seq)
            loss = compute_loss(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        train_accuracy.update_state(labels, predictions)
        return loss

    def test_steps(inputs):
        text_seq, labels = inputs
        predictions = model(text_seq)
        loss = loss_object(labels, predictions)

        test_loss.update_state(loss)
        test_accuracy.update_state(labels, predictions)

In [71]:
model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 1024)]            0         
                                                                 
 encoder_11 (Encoder)        (None, 1024, 300)         31652828  
                                                                 
 fully_connected_layer_2 (Fu  (None, 1)                489985    
 llyConnectedLayer)                                              
                                                                 
Total params: 32,142,813
Trainable params: 2,373,213
Non-trainable params: 29,769,600
_________________________________________________________________


In [72]:
NUM_EPOCHS = 5
with strategy.scope():
    for epoch in range(NUM_EPOCHS):
        epoch_start = datetime.now()
        total_loss = 0.0
        num_batches = 0

        # Training Loop
        for x in train_dataset: 
            total_loss += distributed_training_step(x)
            num_batches += 1

        train_loss = total_loss / num_batches

        # Testing Loop
        for x in val_dataset:
            distributed_test_step(x)

        epoch_end = datetime.now()
        
        template = ("Epoch {}, Loss: {:.2f}, Accuracy: {:.2f}, Test Loss: {:.2f}, Test Accuracy: {:.2f}, \t Elapsed Time: {}")

        print(template.format(
            epoch + 1,
            train_loss,
            train_accuracy.result() * 100,
            test_loss.result() / strategy.num_replicas_in_sync,
            test_accuracy.result() * 100,
            (epoch_end - epoch_start).seconds
        ))

        test_loss.reset_states()
        train_accuracy.reset_states()

        test_accuracy.reset_states()

PerReplica:{
  0: Tensor("output_0_shard_0:0", shape=(), dtype=float32),
  1: Tensor("output_0_shard_1:0", shape=(), dtype=float32),
  2: Tensor("output_0_shard_2:0", shape=(), dtype=float32),
  3: Tensor("output_0_shard_3:0", shape=(), dtype=float32),
  4: Tensor("output_0_shard_4:0", shape=(), dtype=float32),
  5: Tensor("output_0_shard_5:0", shape=(), dtype=float32),
  6: Tensor("output_0_shard_6:0", shape=(), dtype=float32),
  7: Tensor("output_0_shard_7:0", shape=(), dtype=float32)
}
PerReplica:{
  0: Tensor("output_0_shard_0:0", shape=(), dtype=float32),
  1: Tensor("output_0_shard_1:0", shape=(), dtype=float32),
  2: Tensor("output_0_shard_2:0", shape=(), dtype=float32),
  3: Tensor("output_0_shard_3:0", shape=(), dtype=float32),
  4: Tensor("output_0_shard_4:0", shape=(), dtype=float32),
  5: Tensor("output_0_shard_5:0", shape=(), dtype=float32),
  6: Tensor("output_0_shard_6:0", shape=(), dtype=float32),
  7: Tensor("output_0_shard_7:0", shape=(), dtype=float32)
}


ResourceExhaustedError: ignored

In [57]:
te = None
lab = None
for text, label in train_dataset.take(1):
    te = text
    lab = label

In [59]:
lab

<tf.Tensor: shape=(128,), dtype=int64, numpy=
array([1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1])>